# VectorDB Searching Strategy

## 학습 목표

1. Qdrant Named Vectors를 활용한 Dense/Sparse 동시 저장 전략을 이해합니다
2. Prefetch + RRF Fusion을 사용한 하이브리드 검색을 구현할 수 있습니다
3. Dense 임베딩 모델 선택 (Cloud vs TEI)을 적용할 수 있습니다
4. INT8 양자화 및 HNSW 최적화로 프로덕션 성능을 개선할 수 있습니다

In [ ]:
# ============================================================================
# 환경 확인 및 패키지 설치
# ============================================================================
import sys

# Python 버전 확인
python_version = sys.version_info
print(f"Python 버전: {python_version.major}.{python_version.minor}.{python_version.micro}")

if python_version < (3, 10):
    raise RuntimeError("❌ Python 3.10 이상 필요합니다.")

# LangChain 1.0+ 설치
print("\nLangChain 1.0+ 설치 중...")
%pip install -qU langchain langgraph langchain-community

print("LangChain 통합 패키지 설치 중...")
%pip install -qU langchain-openai langchain-qdrant

print("검색 및 유틸리티 설치 중...")
%pip install -qU python-dotenv rank-bm25 kiwipiepy pandas


In [2]:
import json
import os
import sys
import time
import warnings
from collections import Counter
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd
from dotenv import load_dotenv

# Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance,
    FieldCondition,
    Filter,
    MatchValue,
    PointStruct,
    QueryRequest,
    SearchRequest,
    SparseIndexParams,
    SparseVector,
    SparseVectorParams,
    VectorParams,
)

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", module="pydantic")

# ----------------------------------------------------------------------------
# 1. 프로젝트 경로 설정
# ----------------------------------------------------------------------------

NOTEBOOK_DIR = Path.cwd()
DAY2_ROOT = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == "notebook_outputs" else NOTEBOOK_DIR
PROJECT_ROOT = DAY2_ROOT.parent

for path in [DAY2_ROOT]:
    if path.exists() and str(path) not in sys.path:
        sys.path.insert(0, str(path))

print("프로젝트 경로 설정 완료:")
print(f"  - Day2 Root: {DAY2_ROOT}")
print(f"  - Project Root: {PROJECT_ROOT}")

# ----------------------------------------------------------------------------
# 2. 환경 변수 로드
# ----------------------------------------------------------------------------

env_paths = [
    DAY2_ROOT / ".env",
    PROJECT_ROOT / ".env",
]

env_loaded = False
for env_path in env_paths:
    if env_path.exists():
        load_dotenv(env_path, override=True)
        print(f"\n.env 파일 로드: {env_path}")
        env_loaded = True
        break

if not env_loaded:
    print("\n.env 파일을 찾을 수 없습니다.")

프로젝트 경로 설정 완료:
  - Day2 Root: /Users/jhj/Desktop/sds_class/Day2/notebooks
  - Project Root: /Users/jhj/Desktop/sds_class/Day2

.env 파일 로드: /Users/jhj/Desktop/sds_class/Day2/.env


In [ ]:
# ----------------------------------------------------------------------------
# OpenAI / OpenRouter 모델 초기화 헬퍼
# ----------------------------------------------------------------------------
from typing import Literal

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()


def _resolve_api_context() -> tuple[str, str]:
    """선택된 API 키와 베이스 URL 정보를 반환합니다."""
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        raise RuntimeError("OPENROUTER_API_KEY가 필요합니다.")

    base_url = os.getenv("OPENROUTER_API_BASE") or "https://openrouter.ai/api/v1"

    return (api_key, base_url)


def create_openrouter_llm(
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.3,
    max_tokens: int | None = None,
    **kwargs: object,
) -> ChatOpenAI:
    """OpenAI 호환 LLM 생성 헬퍼.

    Args:
        model: 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/gpt-4o, anthropic/claude-3-sonnet, google/gemini-pro)
        temperature: 생성 온도 (0.0-2.0)
        max_tokens: 최대 생성 토큰 수

    Returns:
        ChatOpenAI: 설정된 LLM 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    openai_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "temperature": temperature,
        "max_retries": 3,
        "timeout": 60,
        **kwargs,
    }
    if max_tokens is not None:
        openai_kwargs["max_tokens"] = max_tokens
    if base_url:
        openai_kwargs["base_url"] = base_url
    return ChatOpenAI(**openai_kwargs)


def create_embedding_model(
    model: str = "openai/text-embedding-3-small",
    **kwargs,
) -> OpenAIEmbeddings:
    """OpenAI 호환 임베딩 모델 생성.

    Args:
        model: 임베딩 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/text-embedding-3-small, openai/text-embedding-3-large)
        **kwargs: 추가 파라미터 (encoding_format 등은 model_kwargs로 전달됨)

    Returns:
        OpenAIEmbeddings: 설정된 임베딩 모델 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    # 전달받은 kwargs에서 model_kwargs로 전달할 파라미터 분리
    # encoding_format, extra_headers 등은 model_kwargs로 전달
    model_kwargs: dict = {}
    embedding_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "show_progress_bar": True,
        "skip_empty": True,
    }

    # 전달받은 kwargs 처리
    for key, value in kwargs.items():
        # OpenRouter API 특정 파라미터는 model_kwargs로 전달
        if key in ("encoding_format"):
            model_kwargs[key] = value
        else:
            # 나머지는 OpenAIEmbeddings에 직접 전달
            embedding_kwargs[key] = value

    if base_url:
        embedding_kwargs["base_url"] = base_url

    # model_kwargs가 있으면 전달
    if model_kwargs:
        embedding_kwargs["model_kwargs"] = model_kwargs

    return OpenAIEmbeddings(**embedding_kwargs)


def create_embedding_model_direct(
    model: str = "qwen/qwen3-embedding-0.6b",
    encoding_format: Literal["float", "base64"] = "float",
    input_text: str | list[str] = "",
    **kwargs,
) -> list[float] | list[list[float]]:
    """OpenAI SDK를 직접 사용하여 임베딩 생성 (encoding_format 지원).

    LangChain의 OpenAIEmbeddings가 encoding_format을 지원하지 않을 때 사용.

    Args:
        model: 임베딩 모델 이름
        encoding_format: 인코딩 형식 ("float")
        input_text: 임베딩할 텍스트 (문자열 또는 문자열 리스트)
        **kwargs: 추가 파라미터

    Returns:
        임베딩 벡터 리스트 (단일 텍스트) 또는 리스트의 리스트 (여러 텍스트)
    """
    from openai import OpenAI

    api_key, base_url = _resolve_api_context()

    client = OpenAI(
        base_url=base_url,
        api_key=api_key,
    )

    # input_text가 비어있으면 kwargs에서 가져오기
    if not input_text:
        input_text = kwargs.get("input", "")

    response = client.embeddings.create(
        model=model,
        input=input_text,
        encoding_format=encoding_format,
    )

    # 단일 텍스트인 경우 첫 번째 임베딩 반환
    if isinstance(input_text, str):
        return response.data[0].embedding
    else:
        # 여러 텍스트인 경우 모든 임베딩 반환
        return [item.embedding for item in response.data]


def get_available_model_types() -> dict[str, list[str]]:
    """OpenRouter에서 사용 가능한 모델 유형을 반환합니다.

    Returns:
        dict[str, list[str]]: 모델 유형별 모델 목록
    """
    return {
        "chat": [
            "openai/gpt-4.1",
            "openai/gpt-4.1-mini",
            "openai/gpt-5",
            "openai/gpt-5-mini",
            "anthropic/claude-sonnet-4.5",
            "anthropic/claude-haiku-4.5",
            "google/gemini-2.5-flash-preview-09-2025",
            "google/gemini-pro-2.5",
            "x-ai/grok-4-fast",
            "moonshotai/kimi-k2-thinking",
            "liquid/lfm-2.2-6b",
            "z-ai/glm-4.6",
        ],
        "embedding": [
            "openai/text-embedding-3-small",
            "openai/text-embedding-3-large",
            "google/gemini-embedding-001",
            "qwen/qwen3-embedding-0.6b",
            "qwen/qwen3-embedding-4b",
            "qwen/qwen3-embedding-8b",
        ],
    }


embeddings = create_embedding_model()
llm = create_openrouter_llm()

## Document Pre-processing - Data Flow 정리

```
Notebook 01: 문서 변환 & 청킹
  ├─ PDF → 텍스트 변환
  │   ├─ 고급 옵션: OCR 엔진 선택 (RapidOCR)
  │   ├─ 원격 VLM 연동
  │   └─ 멀티모달 내보내기
  ├─ 메타데이터 저장: ../outputs/converted/{basename}_meta.json
  ├─ 텍스트(마크다운) 저장: ../outputs/converted/{basename}.md
  └─ 청킹 전략 적용 (RecursiveCharacterTextSplitter, SemanticChunker 등)

---

Notebook 02 (현재): 벡터 DB 인덱싱 & 검색
  ├─ 변환된 텍스트(마크다운) 로드
  ├─ 임베딩 & Qdrant 인덱싱
  └─ 하이브리드 검색 (Dense + Sparse, Kiwi 형태소 분석기 등)
```


---

# RAG 시스템을 위한 VectorDB 검색 전략
 - Dense 
 - Sparse 
 - **Hybrid(RRF)**

## 검색 전략 비교

| 전략 | 방법 | 장점 | 단점 | 사용 사례 |
|------|------|------|------|----------|
| **Dense Vector** | 임베딩 유사도 | 의미 파악, 동의어 처리 | 정확한 키워드 약함 | 자연어 질문 |
| **Sparse BM25** | 키워드 빈도(TF-IDF 기반 BM25) + Kiwi | 고유명사, 숫자 정확 | 의미 유사도 무시 | 정확한 용어 검색 |
| **Hybrid(RRF)** | Dense + Sparse 결합 | 두 장점 결합 | 복잡도 증가 | **프로덕션 RAG** |


- 각각 따로 해보는건 너무 쉬운 Naive RAG 이고, 입문 레벨에서 충분히 해보셨을테니 저희는 바로 Hybrid(RRF) 를 직접 구현해보는 단계로 넘어갑니다.

## Hybrid Search: RRF (Reciprocal Rank Fusion)

### RRF란?

**Reciprocal Rank Fusion**은 여러 검색 결과를 '순위'를 활용해 결합하는 알고리즘입니다:

#### 동작 원리

```
Dense 검색 결과: [doc_A(rank=1), doc_B(rank=2), doc_C(rank=3)]
Sparse 검색 결과: [doc_B(rank=1), doc_D(rank=2), doc_A(rank=3)]
```

```
RRF 점수 계산:
doc_A: 1/(60+1) + 1/(60+3) = 0.0164 + 0.0159 = 0.0323
doc_B: 1/(60+2) + 1/(60+1) = 0.0161 + 0.0164 = 0.0325 ← 최고점
doc_C: 1/(60+3) + 0 = 0.0159
doc_D: 0 + 1/(60+2) = 0.0161
```

---

```
최종 순위: [doc_B, doc_A, doc_D, doc_C]
```

---

**공식**: `score(d) = Σ (weight / (k + rank(d)))`
- `k`: RRF 상수 (기본 60)
- `weight`: 각 검색기의 가중치

---

In [ ]:
# ============================================================================
# RRF 알고리즘 구현
# 항상 기본 내용을 알고 있어야 충분히 응용하고 변형할 수 있습니다.
# ============================================================================

from collections import defaultdict
from typing import Any


def reciprocal_rank_fusion(
    dense_results: list[Any],
    sparse_results: list[Any],
    k: int = 60,
    dense_weight: float = 0.5,
    sparse_weight: float = 0.5,
) -> list[dict[str, Any]]:
    """
    Reciprocal Rank Fusion 알고리즘

    Args:
        dense_results: Dense 검색 결과 (Qdrant 결과)
        sparse_results: Sparse 검색 결과 (BM25 결과)
        k: RRF 상수 (기본 60)
        dense_weight: Dense 검색 가중치
        sparse_weight: Sparse 검색 가중치

    Returns:
        융합된 검색 결과 리스트
    """
    # 문서별 RRF 점수 계산
    rrf_scores = defaultdict(float)
    doc_data = {}  # 문서 ID → 문서 데이터

    # Dense 결과 처리
    for rank, result in enumerate(dense_results, start=1):
        doc_id = result.id
        rrf_scores[doc_id] += dense_weight / (k + rank)
        doc_data[doc_id] = {
            "text": result.payload["text"],
            "metadata": result.payload.get("metadata", {}),
            "dense_score": result.score,
            "dense_rank": rank,
        }

    # Sparse 결과 처리
    for rank, result in enumerate(sparse_results, start=1):
        # BM25 결과는 인덱스로 식별 (chunks_data의 인덱스)
        doc_id = rank - 1  # 임시: 인덱스를 doc_id로 사용
        rrf_scores[doc_id] += sparse_weight / (k + rank)

        if doc_id not in doc_data:
            doc_data[doc_id] = {
                "text": result["document"],
                "metadata": result.get("metadata", {}),
                "sparse_score": result["score"],
                "sparse_rank": rank,
            }
        else:
            doc_data[doc_id]["sparse_score"] = result["score"]
            doc_data[doc_id]["sparse_rank"] = rank

    # 점수 기준 정렬
    sorted_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)

    # 결과 생성
    results = []
    for doc_id, rrf_score in sorted_docs:
        data = doc_data[doc_id]
        results.append(
            {
                "doc_id": doc_id,
                "text": data["text"],
                "metadata": data["metadata"],
                "rrf_score": rrf_score,
                "dense_score": data.get("dense_score", 0),
                "sparse_score": data.get("sparse_score", 0),
                "dense_rank": data.get("dense_rank", None),
                "sparse_rank": data.get("sparse_rank", None),
            }
        )

    return results